# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO

For this exercise, we will use the well-known MNIST digit data. To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
import datetime
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
now

<function datetime.now(tz=None)>

In [4]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [5]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [6]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [7]:
K.image_data_format()

'channels_last'

In [8]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training
import tensorflow as tf

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = tf.keras.utils.to_categorical(train[1], num_classes)
    y_test = tf.keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [9]:
import numpy as np
a=np.array([ [  [[1,2]] ] ])
a.shape

(1, 1, 1, 2)

In [10]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [11]:
# x_train_gte5.shape
# x_test_gte5.shape

# x_train_gte5.shape
# x_train = x_train_gte5[0].reshape((x_train_gte5[0].shape[0],) + (28, 1))
# x_train_gte5[0].shape

In [12]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [13]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [14]:
# feature_layers = [
#     Conv2D(filters, kernel_size, padding='valid', input_shape=input_shape),
#     Activation('relu'),
#     Conv2D(filters, kernel_size),
#     Activation('relu'),
#     MaxPooling2D(pool_size=pool_size),
#     Dropout(0.25),
#     Flatten(),
# ]
# classification_layers = [
#     Dense(128),
#     Activation('relu'),
#     Dropout(0.5),
#     Dense(num_classes),
#     Activation('softmax')
# ]

In [15]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [16]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [17]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 23s 97ms/step - loss: 1.6064 - accuracy: 0.2309 - val_loss: 1.5843 - val_accuracy: 0.4114
Epoch 2/5
230/230 [==============================] - 28s 121ms/step - loss: 1.5818 - accuracy: 0.3006 - val_loss: 1.5559 - val_accuracy: 0.5853
Epoch 3/5
230/230 [==============================] - 29s 127ms/step - loss: 1.5531 - accuracy: 0.3818 - val_loss: 1.5258 - val_accuracy: 0.6855
Epoch 4/5
230/230 [==============================] - 33s 142ms/step - loss: 1.5255 - accuracy: 0.4444 - val_loss: 1.4928 - val_accuracy: 0.7324
Epoch 5/5
230/230 [==============================] - 27s 117ms/step - loss: 1.4932 - accuracy: 0.5018 - val_loss: 1.4556 - val_accuracy: 0.7562
Training time: 0:02:20.399851
Test score: 1.4556121826171875
Test accuracy: 0.7562230229377747


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [18]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [20]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 10s 38ms/step - loss: 1.5690 - accuracy: 0.2842 - val_loss: 1.5337 - val_accuracy: 0.4489
Epoch 2/5
240/240 [==============================] - 6s 24ms/step - loss: 1.5257 - accuracy: 0.3718 - val_loss: 1.4874 - val_accuracy: 0.5437
Epoch 3/5
240/240 [==============================] - 7s 29ms/step - loss: 1.4831 - accuracy: 0.4486 - val_loss: 1.4424 - val_accuracy: 0.6423
Epoch 4/5
240/240 [==============================] - 7s 30ms/step - loss: 1.4413 - accuracy: 0.5204 - val_loss: 1.3982 - val_accuracy: 0.7285
Epoch 5/5
240/240 [==============================] - 7s 30ms/step - loss: 1.4018 - accuracy: 0.5767 - val_loss: 1.3544 - val_accuracy: 0.7865
Training time: 0:00:36.967436
Test score: 1.3544282913208008
Test accuracy: 0.7865343689918518


In [21]:
def test_model_sample(test, num_classes):
#     x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
#     x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
#     x_train /= 255
    x_test /= 255
#     print('x_train shape:', x_train.shape)
#     print(x_train.shape[0], 'train samples')
#     print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
#     y_train = tf.keras.utils.to_categorical(train[1], num_classes)
    y_test = tf.keras.utils.to_categorical(test[1], num_classes)
    
    return x_test, y_test

In [23]:
test_sample = test_model_sample((x_test_gte5, y_test_gte5), num_classes)
score = model.evaluate(test_sample[0], test_sample[1], verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 1.4457767009735107
Test accuracy: 0.6445176005363464


In [25]:
test_sample2 = test_model_sample((x_test_lt5, y_test_lt5), num_classes)
score2 = model.evaluate(test_sample2[0], test_sample2[1], verbose=0)
print('Test score:', score2[0])
print('Test accuracy:', score2[1])

Test score: 1.3544282913208008
Test accuracy: 0.7865343689918518


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.

In [ ]:
# Create layers and define the model as above
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]
model2 = Sequential(feature_layers2 + classification_layers2)
model2.summary()

In [ ]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

In [ ]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [ ]:
model2.summary()

In [ ]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

---
### Machine Learning Foundation (C) 2020 IBM Corporation